## PointNet Training Module

##### Mike Pieschl

This module implements the original PointNet model described in PointNet: Deep Leanring on Point Sets for 3D Classification and Segmentation (2017)

In [1]:
import copy
import os
import trimesh
import glob
import json
import pickle
import tf2onnx
import numpy as np
import pandas as pd
import open3d as o3d
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import onnxruntime as ort
import onnx

from plotly.subplots import make_subplots
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from ipywidgets import interact
from IPython.display import clear_output
from tqdm import tqdm

import importlib
import PointNetSegmentation
import PointCloudSet
import mat_ops

importlib.reload(PointNetSegmentation)
importlib.reload(PointCloudSet)
importlib.reload(mat_ops)

2025-11-17 10:46:03.999470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<module 'mat_ops' from '/mnt/c/AFIT/AAR/4. Code/point_cloud_analysis/mat_ops.py'>

In [2]:
USE_SAVED_DATA = True
SAVE_DATA = True
SHOW_DISPLAYS = True

BATCH_SIZE = 32
EPOCHS = 3
PATIENCE = 50
INPUT_SIZE = 4096
LEARNING_RATE = 0.0001
LR_DECAY_STEPS = 7000
LR_DECAY_RATE = 0.7

In [3]:
MODEL_PATH = 'models/'
MESH_PATH = 'mesh/'
FIGURE_PATH = 'figures/'
DATA_PATH = 'data/'
PALINDROME_DATA_PATH = '/mnt/c/repos/aburn/usr/hub/palindrome_playground/DataCollect/'
RANDOM_SEED = 42

MODEL_NAME = '_test_pointnet_softmax'
PC_NAME = 'collect_2025.Nov.12_16.57.13.0087094.UTC'

class_labels = ['kc46']
part_labels = ['fuselage', 'left_engine', 'right_engine', 'left_wing', 'right_wing', 'left_hstab', 'right_hstab', 'vstab', 'left_boom_stab', 'right_boom_stab', 'boom_wing', 'boom_hull', 'boom_hose']

In [4]:
physical_devices = tf.config.experimental.list_physical_devices( 'GPU' )
if( len( physical_devices ) > 0 ): 
    print( 'GPUs Available: ', len( physical_devices ) )
    tf.config.experimental.set_memory_growth( physical_devices[0], True )
else:   print( "No GPUs available." )

GPUs Available:  1


#### Parse AftrBurner output

In [5]:
# pc = PointCloudSet.PointCloudSet(one_hot = True,
#                                  class_labels = class_labels, 
#                                  part_labels = part_labels, 
#                                  pretrain_tnet = False, 
#                                  network_input_width = INPUT_SIZE,
#                                  batch_size = 8,
#                                  rand_seed = RANDOM_SEED)
# pc.build_from_aftr_output(f'{PALINDROME_DATA_PATH}collect_2025.Nov.12_16.57.13.0087094.UTC')
# pc.get_info()
# with open(f'{DATA_PATH}collect_2025.Nov.12_16.57.13.0087094.UTC.pkl', 'wb') as p:
#     pickle.dump(pc, p)

#### Load PointCloudSet using pickle

In [6]:
pc = None
with open(f'{DATA_PATH}{PC_NAME}.pkl', 'rb') as p:
    pc = pickle.load(p)

assert pc != None, 'PointCloudSet failed to load.'

#### Define Training Networks

In [7]:
def train_save_so3(point_cloud: PointCloudSet.PointCloudSet, name: str):    
    so3_net = PointNetSegmentation.TNetRegressor(add_regularization = False)
    so3_net.build(input_shape = (None, INPUT_SIZE, 3))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        LEARNING_RATE,
        decay_steps = LR_DECAY_STEPS,
        decay_rate = LR_DECAY_RATE,
        staircase = False
    )

    optimizer = keras.optimizers.Adam(
        learning_rate = lr_schedule
    )

    so3_net.compile(
        optimizer = optimizer,
        loss = 'mse',
        metrics = [keras.metrics.RootMeanSquaredError()]
    )

    early_stopping = EarlyStopping(
        monitor = 'val_loss',
        patience = PATIENCE,
        verbose = 1,
        restore_best_weights = True
    )

    train = point_cloud.get_train_tnet_set()
    val = point_cloud.get_val_tnet_set()

    history = so3_net.fit(x = train, epochs = EPOCHS, verbose = 1, validation_data = val, callbacks = [early_stopping])
    so3_net.input_transform.save_weights(f'{MODEL_PATH}{name}_tnet.weights.h5')

    with open(f'{MODEL_PATH}{name}_tnet_history.json', 'w') as j:
        json.dump({
            'loss': history.history['loss'],
            'val_loss': history.history['val_loss'],
            'root_mean_squared_error': history.history['root_mean_squared_error'],
            'val_root_mean_squared_error': history.history['val_root_mean_squared_error'],
        }, j)

    return so3_net

def load_so3_net(name: str):
    so3_net = PointNetSegmentation.TNetRegressor(add_regularization = False)
    so3_net.build((None, INPUT_SIZE, 3))
    so3_net.input_transform.load_weights(f'{MODEL_PATH}{name}_tnet.weights.h5', skip_mismatch = False)
    return so3_net

def train_save_pointnet_segmentater(point_cloud: PointCloudSet.PointCloudSet, name: str, use_pretrained_tnet: bool = False):    
    model = PointNetSegmentation.PointNetSegmentation(output_width = len(part_labels))
    model.build(input_shape = (None, INPUT_SIZE, 3))

    if(use_pretrained_tnet):
        model.input_transform.load_weights(f'{MODEL_PATH}{name}_tnet.weights.h5', skip_mismatch = False)
        model.input_transform.trainable = False

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        LEARNING_RATE,
        decay_steps = LR_DECAY_STEPS,
        decay_rate = LR_DECAY_RATE,
        staircase = False
    )

    optimizer = keras.optimizers.Adam(
        learning_rate = lr_schedule
    )

    model.compile(
        optimizer = optimizer,
        loss = keras.losses.CategoricalCrossentropy(),
        metrics = [keras.metrics.CategoricalAccuracy()]
    )

    early_stopping = EarlyStopping(
        monitor = 'val_loss',
        patience = PATIENCE,
        verbose = 1,
        restore_best_weights = True
    )

    train = point_cloud.get_train_seg_set()
    val = point_cloud.get_val_seg_set()

    history = model.fit(x = train, epochs = EPOCHS, verbose = 1, validation_data = val, callbacks = [early_stopping])

    model.save(f'{MODEL_PATH}{name}.keras')

    with open(f'{MODEL_PATH}{name}_history.json', 'w') as j:
        json.dump({
            'loss': history.history['loss'],
            'val_loss': history.history['val_loss'],
            'categorical_accuracy': history.history['categorical_accuracy'],
            'val_categorical_accuracy': history.history['val_categorical_accuracy'],
        }, j)

    return model

#### (Model40 Dataset for Verification)

In [8]:
# DATA_DIR = tf.keras.utils.get_file("modelnet.zip", "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip", extract = True)
# DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "modelnet_extracted/ModelNet10")

# def parse_dataset(num_points = INPUT_SIZE):

#     train_points = []
#     train_labels = []
#     test_points = []
#     test_labels = []
#     class_map = {}
#     folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))
    
#     for i, folder in tqdm(enumerate(folders)):
#         print(f"Processing class {os.path.basename(folder)}")

#         # Store folder name with ID so we can retrieve later
#         class_map[i] = folder.split("/")[-1]

#         # Gath all files in folder
#         train_files = glob.glob(os.path.join(folder, "train/*"))
#         test_files = glob.glob(os.path.join(folder, "test/*"))

#         for f in train_files:
#             train_points.append(trimesh.load(f).sample(num_points))
#             train_labels.append(i)

#         for f in test_files:
#             test_points.append(trimesh.load(f).sample(num_points))
#             test_labels.append(i)
    
#     return (np.array(train_points),
#             np.array(test_points),
#             np.array(train_labels),
#             np.array(test_labels),
#             class_map)

# def augment(points, label):
#     # Jitter points
#     points += tf.random.uniform(points.shape, -0.005, 0.005, dtype = tf.float64)
    
#     # Shuffle points
#     points = tf.random.shuffle(points)

#     return points, label

# train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(1024)

# train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))

# train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
# test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

#### Train Models

In [9]:
if(pc != None):    
    model = train_save_pointnet_segmentater(pc, f'{pc.get_description()}{MODEL_NAME}', use_pretrained_tnet = False)
    loss, accuracy = model.evaluate(pc.get_test_seg_set())

    print(loss, accuracy)

I0000 00:00:1763394439.047065   15312 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6078 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:01:00.0, compute capability: 7.5


Training data size:  obs = (6337, 4096, 3) | labels = (6337, 4096) 
Validation data size:  obs = (1268, 4096, 3) | labels = (1268, 4096) 
Epoch 1/3
Training: True
Training: True


2025-11-17 10:49:35.679471: I external/local_xla/xla/service/service.cc:163] XLA service 0x75ee8000ba50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-17 10:49:35.679509: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Quadro RTX 4000, Compute Capability 7.5
2025-11-17 10:49:35.971130: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-17 10:49:37.604150: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91501
2025-11-17 10:49:45.337297: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng0{} for conv (f32[1088,512,1,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[1088,8,4096,1]{3,2,1,0}, f32[512,8,4096,1]{3,2,1,0}), window={size=4096x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id"

793/793 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - categorical_accuracy: 0.3651 - loss: 2.0678Training: False
793/793 ━━━━━━━━━━━━━━━━━━━━ 171s 160ms/step - categorical_accuracy: 0.3653 - loss: 2.0673 - val_categorical_accuracy: 0.5110 - val_loss: 3.1302
Epoch 2/3
793/793 ━━━━━━━━━━━━━━━━━━━━ 116s 146ms/step - categorical_accuracy: 0.6460 - loss: 1.1450 - val_categorical_accuracy: 0.5921 - val_loss: 1.4326
Epoch 3/3
793/793 ━━━━━━━━━━━━━━━━━━━━ 125s 157ms/step - categorical_accuracy: 0.7160 - loss: 0.9150 - val_categorical_accuracy: 0.6444 - val_loss: 1.2195
Restoring model weights from the end of the best epoch: 3.
106/106 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - categorical_accuracy: 0.6333 - loss: 1.1714
1.1164791584014893 0.640175998210907


In [10]:
custom_objects = {
    "PointNetSegmentation": PointNetSegmentation.PointNetSegmentation,
    "TNet": PointNetSegmentation.TNet,
    "ConvLayer": PointNetSegmentation.ConvLayer,
    "DenseLayer": PointNetSegmentation.DenseLayer
}

test_reload_model = tf.keras.models.load_model(
    f'{MODEL_PATH}{MODEL_NAME}.keras',
    custom_objects = custom_objects
)

test_reload_model.compile()
test_reload_model.build(input_shape = (None, INPUT_SIZE, 3))
test_reload_model.summary()

loss, accuracy = test_reload_model.evaluate(pc.get_test_seg_set())

/mnt/c/AFIT/AAR/4. Code/point_cloud_analysis/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 67 variables whereas the saved optimizer has 127 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: You cannot add new elements of state (variables or sub-layers) to a layer that is already built. All state must be created in the `__init__()` method or in the `build()` method.

In [ ]:
test_case = np.expand_dims(pc.get_raw_test_set()['observations'][0], axis = 0)
test_labels = pc.get_raw_test_set()['part_labels'][0]

predict_labels = test_reload_model.predict(test_case).squeeze(axis = 0)
part_labels_np = np.array(part_labels)
print(f"{np.sum(np.array([part_labels[i] == test_labels[i] for i in np.argmax(predict_labels, axis = -1)]))}")

In [ ]:
input_signature = [
    tf.TensorSpec((None, INPUT_SIZE, 3), dtype = tf.float32)
]

onnx_model, _ = tf2onnx.convert.from_keras(
    test_reload_model,
    input_signature = input_signature,
    opset = 13
)

onnx.save(onnx_model, f'{MODEL_PATH}{MODEL_NAME}.onnx')

In [ ]:
print(f"ONNX Runtime Providers: {ort.get_available_providers()}")

session = ort.InferenceSession(
    f'{MODEL_PATH}{MODEL_NAME}.onnx',
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print(f'Input name: {input_name}')
print(f'Output name: {output_name}')

test_inputs_f32 = test_case.astype(np.float32)

logits_output = session.run(
    [output_name],
    {input_name: test_inputs_f32}
)

print(logits_output)

part_labels_np = np.array(part_labels)
output_np = np.squeeze(np.array(logits_output), axis = (0, 1))
print(f"{np.sum(np.array([part_labels[i] == test_labels[i] for i in np.argmax(output_np, axis = -1)]))}")